In [ ]:


from uuid import uuid4, UUID
from collections import namedtuple, defaultdict
from enum import Enum
from itertools import product

class StoreException(UserWarning):
    """All Exceptions specific to this package for easy filtering."""

class Predicate:
    """See django validators."""
    def validate(self, value):
        return True
    
    @property
    def name(self):
        return self.__class__.__name__

class E:
    """Entity.
    Most entities will be anonymous, which is perfectly fine, while some can have names.
    Entities are the same whenever they have the same id, which is unique.
    These are NOT singletons (which could be conceivable), but due to the UUID used as
    hash, they behave very similar, for instance as keys in dicts.
    
    >>> x = E("hello")
    >>> y = eval(repr(x))
    >>> x == y
    True
    """
    def __init__(self, name=None, id_=None, url=None):        
        
        self.name = None
        
        if name is None:
            self.name = None
        elif not str.isidentifier(name):
            raise StoreException("%s not an identifier."%name)
        else:
            self.name = name
        
        self.id = UUID(id_) if id_ is not None else uuid4()
    
    def __hash__(self):
        return self.id.int
    
    def __str__(self):
        return self.name if self.name is not None else "_" + str(self.id)[:5]
    
    def __repr__(self):
        """Return representation so that e == eval(repr(e))"""
        return ("""E("
                f'''{f"name='{self.name}'," if self.name is not None else ''}'''
                f"id_='{self.id}',
                f"url='{self.url}')"""
               )
    
    def __eq__(self, other):
        return self.id == other.id


SPO = namedtuple("Triplet", "s p o")

class TripleStore:
    """A set of three dicts that work together as one.
    
    Keep in mind that the primary dict - spo - is a Python3 dict,
    which works as an OrderedDict that remembers insertion order.
    It might be a good idea to have the other two dicts as weakrefs,
    but that still needs to be figured out.
    
    Subjects need to be unique Entities because
    head = {"eye": {"side": {"left}}, "eye": {"side": "right"}}
    naturally would only count 1 eye instead of two different ones.
    Please note that store.add returns the entity (or list of entities for
    store.add_all) that was the target of the operation to simplify workflow and tests. 
    Since these return values matter in doctests, we assign them to dummy variables.
    
    >>> body = TripleStore()
    >>> class Side(Predicate):
    >>>    pass
    >>> class Is_a(Predicate):
    >>>    pass
    >>> class Name(Predicate):
    >>>    pass
    
    side = Side()
    is_a = Is_a()
    
    e1 = E("eye")
    
    >>> body.add({e1:side})
    
    >>> len(list(body[:P.name:"eye"]))
    2
    
    >>> fingers = "thumb index middle ring pinky".split()
    >>> sides = ["left", "right"]
    
    >>> body.add_all({P.name:fingers, P.side:sides, P.is_a:["finger"]})
    >>> len(list(body[:P.is_a:"finger"]))
    10
    
    There are ways for manipulating entries:
    
    >>> x = body.get({P.name:"eye", P.side:"left"})
    
    # >>> body[x] == {x: {"name":{"eye"}, "side":{"left"}}}
    # >>> body[x:"color"] = "blue"
    
    #>>> body[x:"color"]
    #"blue"
    """
    
    def __init__(self):
        # weakrefs would be nice to have here, but it can't be guaranteed
        # that these are the only references to entities, which makes weakrefs 
        # quite dangerous.
        self._spo = {}  # {subject: {predicate: set([object])}}
        self._pos = {}  # {predicate: {object: set([subject])}}
        self._osp = {}  # {object: {subject, set([predicate])}}
        # sic!
        self._checks = defaultdict(lambda: lambda o: True)
    
    def set_check(self, p, func):
        self._checks[p] = func
    
    def __setitem__(self, key, value):
        def add2index(index, a, b, c):
            if a not in index:
                index[a] = {b: set([c])}
            else:
                if b not in index[a]:
                    index[a][b] = set([c])
                else:
                    index[a][b].add(c)
        
        if not isinstance(key, slice):
            raise StoreException("must be store[s:p] = o")
        elif key.step is not None:
            raise StoreException("slice must be two-part, not three")
        else:
            s = key.start if isinstance(key.start, E) else E(key.start)
            p = key.stop
            if not isinstance(p, BasePredicate):
                raise StoreException("%s must be a Predicate()."%p)
            o = value
            if not self._checks[p](o):
                raise StoreException("%s does not match the set criteria for this predicate"%o)
            add2index(self._spo, s, p, o)
            add2index(self._pos, p, o, s)
            add2index(self._osp, o, s, p)
        
    def __getitem__(self, key):
        """
        Return iterator over triplets directly as result.
        
        This is a mechanism inspired by the brilliant way numpy handles
        arrays and its vectorization methods. 
        Another way of querying is the .get() method inspired by django which
        returns a Query object, representing a view on the dictionary to be evaluated lazily.
        
        The case dictionary here is an invention of mine after 
        researching alternatives for page-long if-clauses that 
        are detrimental to readability.
        It works like this: 
        1) extract s, p and o from the given store[s:p:o] call
        2) go to the bottom of the function and check which parts are given
        3) pass the resulting tuple into the case dict as key and execute the stored func
        4) since the anonymous funcs are closures with access to the local variables,
           they easily can build generators with those and return them.
        """
        
        # we query for an entity directly
        if not isinstance(key, slice):
            assert isinstance(key, E)
            # This is a bit tricky because P.thing is not a valid identifier.
            # This becomes a problem if different sets of predicates are defined
            # as namespaces with conflicting names.
            # In this case, we resolve the conflict the django-way by mangling
            # predicates to P__thing.
            properties = self._spo[key]
            try:
                return namedtuple(f"{str(key)}", 
                              [p.name for p in properties.keys()])(
                                **{k.name:v for k, v in properties.items()})
            except ValueError:
                return namedtuple(f"{str(key)}", 
                        [f"{p.__class__.__name__}__{p.name}"
                         for p in properties.keys()])(
                    **{f"{k.__class__.__name__}__{k.name}":v 
                         for k, v in properties.items()})
        else:
            s, p, o = key.start, key.stop, key.step
            
        case = {(True, True, True): lambda: {(s, p, o) 
                                             for x in (1,) 
                                             if o in self._spo[s][p]},
              (True, True, False): lambda: {(s, p, OBJ) 
                                            for OBJ in self._spo[s][p]},
              (True, False, True): lambda: {(s, PRED, o) 
                                            for PRED in self._osp[o][s]},
              (True, False, False): lambda: {(s, PRED, OBJ) 
                                             for PRED, objset in self._spo[s].items()
                                             for OBJ in objset},
              (False, True, True): lambda: {(SUB, p, o) 
                                            for SUB in self._pos[p][o]},
              (False, True, False): lambda: {(SUB, p, OBJ) 
                                             for OBJ, subset in self._pos[p].items() 
                                             for SUB in subset},
              (False, False, True): lambda: {(SUB, PRED, o) 
                                             for SUB, predset in self._osp[o].items()
                                             for PRED in predset},
              (False, False, False): lambda: {(SUB, PRED, OBJ) 
                                             for SUB, predset in self._spo.items()
                                             for PRED, objset in predset.items()
                                             for OBJ in objset}
             }
        # .get with default won't work here because any of the 
        # dicts may throw KeyError
        try:
            return case[(s is not None,  p is not None, o is not None)]()
        except KeyError:
            return ()
    
    def __len__(self):
        return len(self._spo)
    
    def __iter__(self):
        """Return the same iterator as Store[::] for convenience."""
        return ((SUB, PRED, OBJ) for SUB, predset in self._spo.items()
                                for PRED, objset in predset.items()
                                for OBJ in objset)
    
    def add(self, d, s=None):
        """Convenience method to add a new item to the store."""
        s = s if s is not None else E()
        for p, o in d.items():
            self[s:p] = o
        return s
    
    def add_all(self, d, list_of_s=None):
        """Add all combinations of predicate:object to the store.
        
        From a dict of key:[list of values] we produce a list of all combinations
        of [(key1,value1), (key1,value2)] from which we can build a new dict
        to pass into self.add as parameters.
        
        The subjects (param S) is a list of entities that all these combinations 
        will be added to.
        
        >>> body = TripleStore()
        >>> P = BasePredicate("P", "name has side")
        >>> torso = E("torso")
        >>> body.add({P.name:"torso"}, s=torso) == torso
        True
        >>> legs = body.add_all({P.name:["leg"], P.side:["left", "right"]})
        >>> body.add_all({P.has:legs}, list_of_s=[torso]) == [torso]
        True
        >>> body[torso].has == set(legs)
        True
        >>> body.add_all({P.has:["muscle"]}, list_of_s=legs) == legs
        True
        >>> x = body.get({P.name:"leg", P.side:"left"})
        >>> "muscle" in body[x].has
        True
        """

        # simple case: no subjects were given, so we create one for each combination
        if list_of_s is None:
            results = []
            combos = product(*[[(k, v) for v in d[k]] for k in d.keys()])
            for c in combos:
                params = {k:v for k, v in c}
                # I'd rather yield, but that'd mean it won't return!
                results.append(self.add(params))
            return results
        else:
            # we have a list of entities, which we use as target for each combination
            for s in list_of_s:
                for p, O in d.items():
                    for o in O:
                        self[s:p] = o
            return list_of_s

    def get(self, clause_dict):
        """Get the item from the store that matches ALL clauses."""
        clauses = list(clause_dict.items())
        # we need to init the results somehow
        k, v = clauses.pop()
        result = {s for s, p, o in self[:k:v]}
        for k, v in clauses:
            result.intersection_update({s for s, p, o in self[:k:v]})
        if len(result) > 1:
            raise AttributeError("More than a single item matches the criteria.")
        elif len(result) == 0:
            return None
        else:
            return result.pop()
    
    def get_all(self, clause_dict):
        """Get all items from the store that match ALL clauses.
        
        The returned set of items can be reused in any way, including combining
        or excluding items from different queries or manually adding items.
        It is necessary to use a dict here in order to make use of Enum.
        """
        clauses = list(clause_dict.items())
        # we need to init the results somehow
        k, v = clauses.pop()
        result = {s for s, p, o in self[:k:v]}
        for k, v in clauses:
            result.intersection_update({s for s, p, o in self[:k:v]})
        return result

    
    def get_last_added(self):
        """Get the item that was last added to the store."""
        return list(self._spo.keys())[-1]

class Query:
    """Class representing a query to the store."""
    def __init__(self, store, triple):
        self.store = store
        self.triple = triple
        
    def __call__(self):
        return store[triple.s: triple.p: triple.o]

In [ ]:
%%writefile store/store.py

from uuid import uuid4, UUID
from collections import namedtuple, defaultdict
from inspect import isclass
from itertools import product, chain
from dataclasses import dataclass
from warnings import warn
from collections.abc import Set
from enum import Enum
from typing import List

# pydantic is pretty cool :o
from pydantic import (DSN, UUID1, UUID3, UUID4, UUID5, BaseModel, DirectoryPath, EmailStr, FilePath, NameEmail,
                      NegativeFloat, NegativeInt, PositiveFloat, PositiveInt, PyObject, UrlStr, condecimal, confloat,
                      conint, constr)

# remove after dev
from pprint import pprint

class StoreException(UserWarning):
    """All Exceptions specific to this package for easy filtering."""
    
class FailedToComply(StoreException):
    pass

class NotFound(StoreException):
    pass

Triple = namedtuple("Triple", "s p o")
    
class Predicate(BaseModel):
    url: UrlStr
        
    
    @property
    def name(self):
        return self.__class__.__name__
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name

class E:
    """Entity.
    Most entities will be anonymous, which is perfectly fine, while some can have names.
    Entities are the same whenever they have the same id, which is unique.
    These are NOT singletons (which could be conceivable), but due to the UUID used as
    hash, they behave very similar, for instance as keys in dicts.
    """
    def __init__(self, name:str=None, id_:str=None, url:str=None):        
        self.url = url
        
        if name is None:
            self.name = None
        elif not str.isidentifier(name):
            raise StoreException("%s not an identifier."%name)
        else:
            self.name = name

        self.id = UUID(id_) if id_ is not None else uuid4()
    
    def __hash__(self):
        return self.id.int
    
    def __str__(self):
        return self.name if self.name is not None else "_" + str(self.id)[:5]
    
    def __repr__(self):
        """Return representation so that e == eval(repr(e))"""
        return (f"""E(\
{f"name='{self.name}', " if self.name is not None else ''}\
id_='{self.id}'\
{f", url='{self.url}'" if self.url is not None else ''})"""
               )
    
    def __eq__(self, other):
        return hash(self) == hash(other)


class TripleStore:
    """A set of three dicts that work together as one.
    
    Keep in mind that the primary dict - spo - is a Python3 dict,
    which works as an OrderedDict that remembers insertion order.
    It might be a good idea to have the other two dicts as weakrefs,
    but that still needs to be figured out.
    
    Subjects need to be unique Entities because
    head = {"eye": {"side": {"left}}, "eye": {"side": "right"}}
    naturally would only count 1 eye instead of two different ones.
    Please note that store.add returns the entity (or list of entities for
    store.add_all) that was the target of the operation to simplify workflow and tests. 
    Since these return values matter in doctests, we assign them to dummy variables.
    """
    
    def __init__(self):
        self._spo = {}  # {subject: {predicate: set([object])}}
        self._pos = {}  # {predicate: {object: set([subject])}}
        self._osp = {}  # {object: {subject, set([predicate])}}
        
    def __getitem__(self, key):
        """
        Return iterator over triplets directly as result.
        
        This is a mechanism inspired by the brilliant way numpy handles
        arrays and its vectorization methods. 
        Another way of querying is the .get() method inspired by django which
        returns a Query object, representing a view on the dictionary to be evaluated lazily.
        
        The case dictionary here is an invention of mine after 
        researching alternatives for page-long if-clauses that 
        are detrimental to readability.
        It works like this: 
        1) extract s, p and o from the given store[s:p:o] call
        2) go to the bottom of the function and check which parts are given
        3) pass the resulting tuple into the case dict as key and execute the stored func
        4) since the anonymous funcs are closures with access to the local variables,
           they easily can build generators with those and return them.
        """
        
        # we query for an entity directly
        if not isinstance(key, slice):
            assert isinstance(key, E) or isinstance(key, Triple)
            # we return a dict of p:o that we can use in set_all, producing a nice symmetry
            # this is in contrast to the slice method, which returns sets of objects
            return self._spo[key]
        else:
            s, p, o = key.start, key.stop, key.step
            
        # Observe that cases with only one False return no Triples but the values themselves
        # this way, the results can be used directly in an assignment.
        case = {(True, True, True): lambda: {Triple(s, p, o) 
                                             for x in (1,) 
                                             if o in self._spo[s][p]},
              (True, True, False): lambda: {OBJ for OBJ in self._spo[s][p]},
              (True, False, True): lambda: {PRED 
                                            for PRED in self._osp[o][s] if PRED in self._osp[o][s]},
              (True, False, False): lambda: {Triple(s, PRED, OBJ) 
                                             for PRED, objset in self._spo[s].items()
                                             for OBJ in objset},
              (False, True, True): lambda: {SUB for SUB in self._pos[p][o]},
              (False, True, False): lambda: {Triple(SUB, p, OBJ) 
                                             for OBJ, subset in self._pos[p].items() 
                                             for SUB in subset},
              (False, False, True): lambda: {Triple(SUB, PRED, o) 
                                             for SUB, predset in self._osp[o].items()
                                             for PRED in predset},
              (False, False, False): lambda: {Triple(SUB, PRED, OBJ) 
                                             for SUB, predset in self._spo.items()
                                             for PRED, objset in predset.items()
                                             for OBJ in objset}
             }
        try:
            return case[(s is not None,  p is not None, o is not None)]()
        except KeyError:
            warn((s, p, o), NotFound)
        
    
    def __len__(self):
        return len(self._spo)
    
    def __iter__(self):
        """Return the same iterator as Store[::] for convenience."""
        return ((SUB, PRED, OBJ) for SUB, predset in self._spo.items()
                                for PRED, objset in predset.items()
                                for OBJ in objset)
    
    def __contains__(self, value):
        s, p, o = value
        try:
            return o in self._spo[s][p]
        except KeyError:
            return False
    
    def add(self, *, s, p, o):
        def add2index(index, a, b, c):
            if a not in index:
                index[a] = {b: set([c])}
            else:
                if b not in index[a]:
                    index[a][b] = set([c])
                else:
                    index[a][b].add(c)
                    
        if s is None or (s, p, o) in self:
            warn(f"{s, p, o}", FailedToComply)
        
        # Subject can be an existing Triple, which allows for softlink-recursion.
        if isinstance(s, Triple):
            if s not in self:
                raise FailedToComply("Specified subject (Triple) was not found in the store.")
        else:
            if not isinstance(s, E):
                raise FailedToComply("Subject is neither a Triple nor an instance of E.", s, type(s))
        
        assert hasattr(p, "name"), "Predicate has no name!"
        assert hasattr(p, "url"), "Predicate has no url!"
        
        if not p.validate(o):
            raise StoreException(f"{o} does not match the criteria for predicate {p}")
        
        add2index(self._spo, s, p, o)
        add2index(self._pos, p, o, s)
        add2index(self._osp, o, s, p)
        return Triple(s, p, o)
        
    def __setitem__(self, key, value):
        assert isinstance(key, slice), "Must be assigned using a slice (ex: Store[:foo:] = 23)."
        assert isinstance(key.stop, Predicate), "Predicate MUST be specified in slice."
        
        p = key.stop
        o = value
        
        if not (isinstance(key.start, E) or isinstance(key.start, Triple)):
            results = []
            S = key.start
            for s in S:
                results.append(self.add(s=s, p=p, o=o))
            return results
        else:
            s = key.start
            return self.add(s=s, p=p, o=o)

    
    def create_subjects_with(self, predobjects):
        """Add all combinations of predicate:object to the store and create new entities for each combo.
        
        From a dict of key:[list of values] we produce a list of all combinations
        of [(key1,value1), (key1,value2)] from which we can build a new dict
        to pass into self.add as parameters.
        
        """
        combinations = product(*[[(k, v) for v in predobjects[k]] for k in predobjects.keys()])
        subjects = []
        # Trick is to create new entities with a sentinel of None so there is an indefinite amount
        for C, s in zip(combinations, iter(E, None)):
            for p, o in C:
                r = self.add(s=s, p=p, o=o)
            subjects.append(s)
        return subjects

    def set_all(self, *, subjects:List[E], predobjects:dict):
        results = []
        for s in subjects:
            for p, O in predobjects.items():
                for o in O:
                    r = self.add(s=s, p=p, o=o)
                    results.append(r)
        return results
    
    def get(self, clause_dict):
        """Get the item from the store that matches ALL clauses."""
        clauses = list(clause_dict.items())
        # we need to init the results somehow
        k, v = clauses.pop()
        result = {s for s in self[:k:v]}
        for k, v in clauses:
            result.intersection_update({s for s in self[:k:v]})
        if len(result) > 1:
            raise AttributeError("More than a single item matches the criteria.")
        elif len(result) == 0:
            return None
        else:
            return result.pop()
    
    def get_all(self, clause_dict):
        """Get all items from the store that match ALL clauses.
        
        The returned set of items can be reused in any way, including combining
        or excluding items from different queries or manually adding items.
        It is necessary to use a dict here in order to make use of Enum.
        """
        clauses = list(clause_dict.items())
        # we need to init the results somehow
        k, v = clauses.pop()
        result = self[:k:v].copy()
        for k, v in clauses:
            result.intersection_update(self[:k:v])
        return result  # Should be QuerySet

    
    def get_last_added(self):
        """Get the item that was last added to the store."""
        return list(self._spo.keys())[-1]
    
    def undo(self):
        raise NotImplementedError
    
    def __delitem__(self, item):
        raise NotImplementedError
        
    def __str__(self):
        return "".join(f"{str(s)} {str(p)} {str(o)}\n" for s, p, o in self)
        
    

@dataclass
class Query:
    """Class representing a query to the store."""
    store: TripleStore
    spo: Triple
        
    def __call__(self):
        return self.store[self.triple.s: self.triple.p: self.triple.o]


class QuerySet:
    def __init__(self, values):
        self.values = values

    def __getattr__(self, name):
        """Get the value of an attribute."""
        if name in self._sets:
            return self._sets[name]
        else:
            raise AttributeError(f"{name} is not a method or attribute of set or ")

In [14]:
from store.store import TripleStore, Predicate, E, Triple
from enum import Enum

side = Enum("side", "left right")

class Side(Predicate):
    value: side

class Is_a(Predicate):
    value: str

class Name(Predicate):
    pass

class Has(Predicate):
    pass

class Person(Predicate):
    pass

class PartOf(Predicate):
    pass

class Destroyed(Predicate):
    pass

destroyed = Destroyed()
has = Has()


body = TripleStore()
hand = E(name="hand")
ring = E(name="ring")

body[hand:has] = ring
assert Triple(hand, has, ring) in body
body[Triple(hand, has, ring):destroyed] = True

from pprint import pprint
pprint(body[::])

{Triple(s=E(name='hand', id_='3241284c-4f76-448c-aa4a-dd3d6a7c62d3'), p=Has, o=E(name='ring', id_='c0e9ddc5-6979-4a72-a63c-e150e0e0224e')),
 Triple(s=Triple(s=E(name='hand', id_='3241284c-4f76-448c-aa4a-dd3d6a7c62d3'), p=Has, o=E(name='ring', id_='c0e9ddc5-6979-4a72-a63c-e150e0e0224e')), p=Destroyed, o=True)}


In [1]:
from store.store import TripleStore, Predicate, E, Triple

body = TripleStore()

class Side(Predicate):
    pass
class Is_a(Predicate):
    pass
class Name(Predicate):
    pass
class Has(Predicate):
    pass
class Person(Predicate):
    pass
class PartOf(Predicate):
    pass

has = Has()
side = Side()
is_a = Is_a()
name = Name()
person = Person()
partof = PartOf()

people = body.create_subjects_with({is_a:["person"], name:["Anna", "Otto"]})
body.create_subjects_with({partof:people,
                           side:["left", "right"], 
                           name:["eye", "arm", "leg", "hand", "foot", "kidney"]})
body.create_subjects_with({name:["brain", "nose", "liver", "heart", "spleen",], is_a:["organ"]})


body[body[:name:"kidney"]:is_a] = "organ"

body.get_all({name:"kidney"}) & body.get_all({side:"left"}) == body.get_all({name:"kidney", side:"left"})

x = body.get_last_added()
d = body[x]

new_spleen = E()

print(body.set_all(predobjects=d, subjects=[new_spleen]))

print(body)

[Triple(s=E(id_='4f764659-5d0e-4db8-a572-dd6c3321e485'), p=Name, o='spleen'), Triple(s=E(id_='4f764659-5d0e-4db8-a572-dd6c3321e485'), p=Is_a, o='organ')]
_db001 Is_a person
_db001 Name Anna
_1c739 Is_a person
_1c739 Name Otto
_16591 PartOf _db001
_16591 Side left
_16591 Name eye
_67269 PartOf _db001
_67269 Side left
_67269 Name arm
_6e4fb PartOf _db001
_6e4fb Side left
_6e4fb Name leg
_158fc PartOf _db001
_158fc Side left
_158fc Name hand
_5254e PartOf _db001
_5254e Side left
_5254e Name foot
_55fb1 PartOf _db001
_55fb1 Side left
_55fb1 Name kidney
_55fb1 Is_a organ
_db711 PartOf _db001
_db711 Side right
_db711 Name eye
_3c004 PartOf _db001
_3c004 Side right
_3c004 Name arm
_62c51 PartOf _db001
_62c51 Side right
_62c51 Name leg
_2962c PartOf _db001
_2962c Side right
_2962c Name hand
_0a5f5 PartOf _db001
_0a5f5 Side right
_0a5f5 Name foot
_19d8a PartOf _db001
_19d8a Side right
_19d8a Name kidney
_19d8a Is_a organ
_446dc PartOf _1c739
_446dc Side left
_446dc Name eye
_8fbf1 PartOf _1c739

In [ ]:
from collections.abc import Set

class QuerySet(Set):
    def __init__(self, *b):
        set()
    
    def __contains__(self, value):
        return value in self.values
    
    def __iter__(self):
        return iter(self.values)
    
    def __len__(self):
        return len(self.values)
    
    def __getattr__(self, name):
        """Get the value of an attribute."""
        if name in self._sets:
            return self._sets[name]
        else:
            raise AttributeError(f"{name} is not a method or attribute of set or ")
            
a = QuerySet(1,2,3)
b = QuerySet(2)


list(b)

lets try to use it the way intented in PL.
Plato's "ideas" are the idea: define ideal things in terms of their parts - definition by as possibility.

no details are defined as long as nobody looks - schroedinger code!
parts are defined as everything that can be disassembled.
as people should be able to define things and propose new assemblies, we start with the format.

In [18]:
import strictyaml

import yaml

s = """
---
# All about the character
name: Ford #Prefect
age: 42
possessions:
- Towel
---
name: Foo Bar
age: 20
"""

for x in yaml.safe_load(s):
    print(x)

ComposerError: expected a single document in the stream
  in "<unicode string>", line 4, column 1:
    name: Ford Prefect
    ^
but found another document
  in "<unicode string>", line 8, column 1:
    ---
    ^

In [23]:
from strictyaml import load, Map, Str, Int, Seq, YAMLError, Optional
import yaml

s = """
# All about the character
name: Ford #Prefect
age: 42
possessions:
    - Towel
"""

schema = Map({"name": Str(), "age": Int(), "possessions": Seq(Str())})

load(s, schema)

YAML(OrderedDict([('name', 'Ford'), ('age', 42), ('possessions', ['Towel'])]))

In [13]:
from datetime import datetime
from typing import List
from pydantic import BaseModel
import yaml

class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None
    friends: List[int] = []
        


external_data = {'id': '123', 'signup_ts': '2017-06-01 12:22', 'friends': [1, '2', b'3',]}
user = User(**external_data)
print(user)
# > User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
print(user.id)
# > 123


s = """
# All about the character
name: Ford #Prefect
age: 42
possessions:
    - Towel
"""

class Character(BaseModel):
    name: str
    age: int
    possessions: List[str]

data = yaml.safe_load(s)
char = Character(**data)
char

User id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
123


<Character name='Ford' age=42 possessions=['Towel']>